# Import Labraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

# Connection To Database

In [ ]:
conn = sqlite3.connect("Upi_Digital_Payment.db")
cursor = conn.cursor()


# Load Dataset


In [ ]:
df = pd.read_csv("upi_digital_payment_advanced.csv")
df.to_sql("")